In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler, LabelEncoder

In [3]:
import nltk
from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.tokenize import word_tokenize

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
from xgboost import XGBClassifier
import xgboost as xgb

In [56]:
datasrc = 'C:/Projects/FCSD/FNOP/old/Testdata_pilot_sample_v2.csv'

In [57]:
type_dict = {'target': 'str', 'prt_num_causl_base_cd': 'str', 'cust_txt': 'str'}
df_src = pd.read_csv(datasrc, engine='python', dtype=type_dict)

In [58]:
df_src.columns

Index(['clm_key', 'cust_txt', 'mdl_yr', 'prt_num_causl_base_cd', 'tis_wsd',
       'milge', 'veh_line_cd', 'eng_cd', 'cust_conc_cd', 'part_desc',
       'target'],
      dtype='object')

In [59]:
df_src.isna().sum()

clm_key                  0
cust_txt                 0
mdl_yr                   0
prt_num_causl_base_cd    0
tis_wsd                  0
milge                    0
veh_line_cd              0
eng_cd                   0
cust_conc_cd             0
part_desc                0
target                   0
dtype: int64

In [75]:
df_src.dtypes

clm_key                    int64
cust_txt                  object
mdl_yr                     int64
prt_num_causl_base_cd     object
tis_wsd                    int64
milge                    float64
veh_line_cd               object
eng_cd                    object
cust_conc_cd              object
part_desc                 object
target                    object
dtype: object

In [76]:
df_src.shape

(1026631, 11)

In [77]:
np.random.seed(47)
df = df_src.sample(n=10000)
df = df[['cust_txt', 'mdl_yr','tis_wsd','milge', 'veh_line_cd', 'eng_cd', 'target']]

In [78]:
### convert model year into string type
df['mdl_yr'] = df['mdl_yr'].astype('object')

In [79]:
### convert 5 columns into one-hot encoding
df_X_dummy = pd.get_dummies(df[['mdl_yr','tis_wsd','milge', 'veh_line_cd', 'eng_cd']])

In [80]:
df_X_dummy.shape

(10000, 140)

In [10]:
from sklearn.model_selection import train_test_split

### use customized text preprocessor

In [82]:
lookup_table = 'C:/Projects/FCSD/FNOP/old/Misspellings_Correction_Synonyms_Lookup_Table_Master.xlsx'
syn_df=pd.read_excel(lookup_table)
syn_df = syn_df[syn_df['Review'] != "D"]
syn_dict=dict(zip(syn_df.misspelled, syn_df.corrected))

In [83]:
def TextPreprocess(text):
        # Avoid mixed data type errors and apply str on all rows
        # convert to lower case
        text = str(text).lower()

        #Remove HTML Tags
        text = re.sub('<[^>]*>', '', text)
        r1 = re.compile('<.*?>')
        text = re.sub(r1, ' ', text)

#         # Remove any token with numbers and/or special characters
#         r2 = re.compile(r'^[0-9\W\_]+$')
#         text = re.sub(r2, ' ', text)
        
        # move parenthesis content to the end of paragraph
        paren_content_extract = re.compile(r'\(([^()]+)\)')
        p_cont = paren_content_extract.findall(text)
        text = re.sub(r'\([^()]*\)', ' ', text)
        text = text + '. '.join(p_cont) + '.'
        
        # hard code to convert abbreviations with \'s
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", "have not ", text)
        text = re.sub(r"can't", "can not ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r"\'scuse", " excuse ", text)
        
        # if a token starts with numerical or # sign, remove it
        text = re.sub('\b[\d#]+\S*\b', ' ', text)
        # if a token ends with numberical, remove it
        text = re.sub(r'\b\S+\d+\b', ' ', text)

#         #Removing Punctuations
#         text = re.sub(r'[?|!|\'|"|#]',r'',text)
#         text = re.sub(r'[.|,|)|(|\|/]',r' ',text)     

        #Removing Punctuations
        text = re.sub(r'[\'|"|#]',r'',text)
        text = re.sub(r'[)|(]',r' ',text) 
        
        # whitespace characters are replaced into space, multiple dots replaced into single dot
        text = re.sub('\s+', ' ', text)
        text = re.sub(r'\.+', '.', text)

        # Tokenize all text data using nltk package
        text = word_tokenize(text)

#         # Remove stop words
#         stop = stopwords.words('english')
#         stop1 = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z', \
#                  'c/s','c/states','c/stated','cu', 'cs','c-s', 'cus','tomer', 'stomer', 'tate', 'cust',  'cust states', \
#                  'advise','nana','NaN','customer','states','rr','nan','wont','ck','check','pls','please', 'client','owner']

#         # Combine default nltk stop list and custom stop list to generate a deduplicated list
#         english_stops = list(set(stop+stop1))

#         text = [item for item in text if item not in english_stops]

#         # Remove special characters occuring as prefixes
#         text = [re.sub(r'^[\W\_]+','',str(i)) for i in text]
#         # Remove special characters occuring as suffixes
#         text = [re.sub(r'[\W\_]+$','',str(i)) for i in text]

#         # Replace special characters with spaces when occuring within the token 
#         #(i.e. not at the beginning or the end) only for tokens greater than 5 characters in length
#         text =[re.sub(r'[\W\_]+', ' ', str(i)) if len(str(i)) > 5 else i for i in text]

#         # Replace special characters with null when occuring within the token 
#         #(i.e. not at the beginning or the end) only for tokens less than 6 characters in length
#         text = [re.sub(r'[\W\_]+', '', str(i)) if len(str(i)) < 6 else i for i in text]

#         # Remove stop words after data cleansing steps to remove any residual stop words
#         text = [item for item in text if item not in english_stops]

#         # remove any numbers after data cleansing steps to remove any residual numeric values
#         r3 = re.compile(r'^[0-9\W\_]+$')
#         text = [i for i in text if not r3.match(i)]

        #Perform synonym lookup to find matching correctly spelled tokens for misspelled tokens
        #spell=SpellChecker() 
        text = [syn_dict.get(str(i), str(i)) for i in text]

        # Convert the list to string before passing to lemmatization module
        text = ' '.join(text)

#         # Perform Lemmatization on the cleaned up data before tfidf vectorization
#         text = lemmatize_with_postag(text)
        text = text.replace(' .', '.').replace(' ?', '?').replace(' !', '!').replace(' ,', ',').replace(' /', '/')
        return(text)

In [84]:
df['process_txt']=df['cust_txt'].apply(lambda x: TextPreprocess(x))

In [85]:
df = df.drop("cust_txt", axis=1)

### use universal sentence encoder to convert text into vectors

In [18]:
### load the model
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [88]:
sentence_embeddings_X = model(df['process_txt']).numpy()

In [89]:
sentence_embeddings_X.shape

(10000, 512)

#### combine numerical, categorical features and doc embedding

In [90]:
combined_X = np.concatenate((df_X_dummy, sentence_embeddings_X), axis=1)

In [91]:
combined_X.shape

(10000, 652)

#### encode y variable

In [62]:
le = LabelEncoder()
le.fit(df['target'])
label_encoded_y = le.transform(df['target'])

In [95]:
len(le.classes_)

87

### split input into train/test

In [92]:
X_train, X_test, y_train, y_test = train_test_split(combined_X, label_encoded_y, test_size=0.2, random_state=42)

### build xgb model

In [94]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

In [96]:
params = {
    "objective": "multi:softprob",
    "num_class": 87,
    ##"eval_metric": "auc",
    "booster": "gbtree",
    "tree_method": "exact",
    "eta": 0.1,
    "max_depth": 4,
    "sub_sample": 1,
    "col_sample_bytree": 1,
    "verbosity": 0,    
    "seed": 11    
}

In [97]:
num_rounds = 50
early_stopping_rounds = 10

In [98]:
eval_set = [(dtrain, 'train'), (dtest, 'eval')]

In [99]:
gbm = xgb.train(params, dtrain, num_rounds, evals=eval_set, early_stopping_rounds=early_stopping_rounds)

[0]	train-mlogloss:3.23890	eval-mlogloss:3.42650
[1]	train-mlogloss:2.87513	eval-mlogloss:3.16072
[2]	train-mlogloss:2.60798	eval-mlogloss:2.97770
[3]	train-mlogloss:2.39526	eval-mlogloss:2.83363
[4]	train-mlogloss:2.21590	eval-mlogloss:2.70369
[5]	train-mlogloss:2.06149	eval-mlogloss:2.59973
[6]	train-mlogloss:1.92620	eval-mlogloss:2.50852
[7]	train-mlogloss:1.80950	eval-mlogloss:2.42996
[8]	train-mlogloss:1.70320	eval-mlogloss:2.35899
[9]	train-mlogloss:1.60707	eval-mlogloss:2.29405
[10]	train-mlogloss:1.51896	eval-mlogloss:2.23556
[11]	train-mlogloss:1.43975	eval-mlogloss:2.18246
[12]	train-mlogloss:1.36613	eval-mlogloss:2.13539
[13]	train-mlogloss:1.29862	eval-mlogloss:2.09161
[14]	train-mlogloss:1.23659	eval-mlogloss:2.05153
[15]	train-mlogloss:1.17911	eval-mlogloss:2.01479
[16]	train-mlogloss:1.12535	eval-mlogloss:1.98037
[17]	train-mlogloss:1.07586	eval-mlogloss:1.94710
[18]	train-mlogloss:1.02878	eval-mlogloss:1.91889
[19]	train-mlogloss:0.98561	eval-mlogloss:1.89000
[20]	train

In [100]:
probs = gbm.predict(xgb.DMatrix(X_test))

In [101]:
probs.shape

(2000, 87)

In [102]:
df_prob = pd.DataFrame(probs, columns=le.classes_)

In [103]:
## find the top3 parts
k = 3

# GET TOP K PREDICTIONS BY PROB - note these are just index
best_n = np.argsort(probs, axis=1)[:,-k:]

# GET CATEGORY OF PREDICTIONS
preds=[[le.classes_[predicted_cat] for predicted_cat in prediction] for prediction in best_n]

preds=[item[::-1] for item in preds]

df_top3 = pd.DataFrame(preds, columns=("Top1", "Top2", "Top3"))
df_top3.head(10)

Top1    Top2    Top3
0  other1  19E616   12029
1  other2  other1   18125
2   18125    8501   3B436
3  other2    8501  other1
4  other1  other2   9D653
5  other1  other2   12029
6  other1    1104  other2
7  19G490  14B205  19E616
8  other2  other1    3504
9  other2   5K483   18125

In [104]:
### print the actual part from test set
y_test_part = le.inverse_transform(y_test)

In [105]:
### top1 accuracy
N = len(y_test)
n_correct = 0
for i in range(N):
    if (df_top3.iloc[i, 0]==y_test_part[i]):
        n_correct += 1
        
print(n_correct/N)

0.6075


In [107]:
### top3 accuracy
n_correct = 0
for i in range(N):
    if (df_top3.iloc[i, 0]==y_test_part[i]) or (df_top3.iloc[i, 1]==y_test_part[i]) or (df_top3.iloc[i, 2]==y_test_part[i]):
        n_correct += 1
        
print(n_correct/N)

0.808
